# Brain parcellation with TorchIO and HighRes3DNet

We are going to perform a full brain parcellation of a 3D T1-weighted MRI using [TorchIO](https://torchio.readthedocs.io/) and a pre-trained [PyTorch](https://pytorch.org/) deep learning model in less than 50 lines of code and less than two minutes.

## TorchIO

TorchIO is a Python package to prepare 3D medical images for deep learning pipelines. Check out the [documentation](https://torchio.readthedocs.io/) and a longer [Colab notebook](https://colab.research.google.com/drive/112NTL8uJXzcMw4PQbUvMQN-WHlVwQS3i) containing many examples.

## HighRes3DNet

HighRes3DNet is a 3D residual network presented by Li et al. in [On the Compactness, Efficiency, and Representation of 3D Convolutional Networks: Brain Parcellation as a Pretext Task](https://link.springer.com/chapter/10.1007/978-3-319-59050-9_28). The authors shared the weights of the model they trained to perform full brain parcellation as in [Geodesic Information Flows: Spatially-Variant Graphs and Their Application to Segmentation and Fusion](https://spiral.imperial.ac.uk/bitstream/10044/1/30755/4/07086081.pdf), also known as GIF parcellation.

The weights were ported from TensorFlow as shown in [this entry to the MICCAI educational challenge 2019](https://github.com/fepegar/miccai-educational-challenge-2019).

First we will install TorchIO and download a couple of useful files:

In [ ]:
!pip install --quiet torchio==0.18.83
!pip install --quiet pandas matplotlib seaborn scikit-image ipywidgets  # for the visualization module
!curl -sS -O https://raw.githubusercontent.com/fepegar/miccai-educational-challenge-2019/master/visualization.py
!curl -sS -O https://raw.githubusercontent.com/fepegar/highresnet/master/GIFNiftyNet.ctbl

     |████████████████████████████████| 170 kB 3.7 MB/s 
     |████████████████████████████████| 48.4 MB 31 kB/s 


In [ ]:
import datetime
from pathlib import Path

import torch
import numpy as np
import torchio as tio
from tqdm.auto import tqdm, trange
import visualization
plot_volume = visualization.plot_volume_interactive
%config InlineBackend.figure_format = 'retina'
torch.manual_seed(20202021)
print('TorchIO version:', tio.__version__)
print('Last run:', datetime.date.today())

torch.set_grad_enabled(False);

TorchIO version: 0.18.83
Last run: 2022-07-18


## Preparing the data

Let's download a T1-weighted MRI hosted on the NiftyNet model zoo:

In [ ]:
root_dir = Path('data')
tio.download.download_and_extract_archive(
    'https://github.com/NifTK/NiftyNetModelZoo/raw/5-reorganising-with-lfs/highres3dnet_brain_parcellation/data.tar.gz',
    root_dir
)
mri_path = list(root_dir.glob('*.nii.gz'))[0]
mri_path

0it [00:00, ?it/s]

PosixPath('data/OAS1_0145_MR2_mpr_n4_anon_sbj_111.nii.gz')

We will now create an instance of [`torchio.ScalarImage`](https://torchio.readthedocs.io/data/image.html#torchio.ScalarImage) and pass it to a [`torchio.Subject`](https://torchio.readthedocs.io/data/subject.html):

In [ ]:
subject_oasis = tio.Subject(t1=tio.ScalarImage(mri_path))
subject_colin = tio.datasets.Colin27()
subject = subject_oasis  # try subject_colin instead!

0it [00:00, ?it/s]

We are going to apply four [preprocessing transforms](https://torchio.readthedocs.io/transforms/preprocessing.html) to our instance of `Subject`:

1. [`ToCanonical`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.ToCanonical) reorients our image using [NiBabel](https://nipy.org/nibabel/) so that it is in [RAS+ orientation](https://nipy.org/nibabel/image_orientation.html)
2. [`Resample`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.Resample) changes the voxels spacing using [SimpleITK](https://simpleitk.org/). This is analogous to using [`torchvision.transforms.Resize`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Resize)
3. [`ZNormalization`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.ZNormalization) generates an image with zero mean and unit variance. Both normalization transforms use the foreground values (computed as the values over the mean intensity) to calculate the corresponding statistics
4. [`Crop`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.Crop) reduces the field of view by removing voxels from the image. This will make our computations slightly faster.

In [ ]:
transforms = [
    tio.ToCanonical(),
    tio.Resample(1),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.Crop((0, 0, 10, 30, 40, 40)),
]
transform = tio.Compose(transforms)
preprocessed = transform(subject)

The transforms are chained together using [`torchio.Compose`](https://torchio.readthedocs.io/transforms/augmentation.html#compose) or [`torchvision.transforms.Compose`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Compose). As you can see, the interface is similar to [`torchvision.transforms`](https://pytorch.org/docs/stable/torchvision/transforms.html). Let's take a look at the input volume:

In [ ]:
plot_volume(subject.t1.numpy().squeeze())

Output()

The labels are incorrect because `plot_volume` expects an image in [RAS+ orientation](https://nipy.org/nibabel/image_orientation.html). It was a good idea to add `ToCanonical` to our list of transforms:

In [ ]:
plot_volume(preprocessed.t1.numpy().squeeze())

Output()

## Pretrained model

We will use the wonderful [PyTorch Hub](https://pytorch.org/hub/) to download the pretrained model from GitHub:

In [ ]:
repo = 'fepegar/highresnet'
model_name = 'highres3dnet'
model = torch.hub.load(repo, model_name, pretrained=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model.to(device).eval();

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/fepegar/highresnet/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/fepegar/highresnet-models/raw/master/highres3dnet_li_parameters-7d297872.pth" to /root/.cache/torch/hub/checkpoints/highres3dnet

Device: cuda


## Inference


In [ ]:
input_tensor = preprocessed.t1.data[None].to(device)  # add batch dim
with torch.autocast(device.type):
    logits = model(input_tensor)
full_volume_output_tensor = logits.argmax(dim=tio.CHANNELS_DIMENSION, keepdim=True).cpu()
plot_volume(
    full_volume_output_tensor.numpy().squeeze(),
    enhance=False,
    colors_path='GIFNiftyNet.ctbl',
)

Output()

Since the image might be too large for the available GPU, we could [perform inference using image patches](https://torchio.readthedocs.io/data/patch_based.html) instead.

We will use a [`GridSampler`](https://torchio.readthedocs.io/data/patch_inference.html#grid-sampler) to extract patches from all the necessary locations in the image. `GridSampler` is a subclass of [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset), so we can easily extract batches of patches using a [`torch.utils.data.Dataloader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataloader).

We will add a small overlap between patches to avoid the border effect.

In [ ]:
patch_overlap = 4
patch_size = 128
grid_sampler = tio.inference.GridSampler(
    preprocessed,
    patch_size,
    patch_overlap,
)
patch_loader = torch.utils.data.DataLoader(grid_sampler)
aggregator = tio.inference.GridAggregator(grid_sampler)
preprocessed.clear_history()  # so that image is not padded at the end

for patches_batch in tqdm(patch_loader, unit='batch'):
    input_tensor = patches_batch['t1'][tio.DATA].to(device)
    locations = patches_batch[tio.LOCATION]
    with torch.autocast(device.type):
        logits = model(input_tensor)
    labels = logits.argmax(dim=tio.CHANNELS_DIMENSION, keepdim=True)
    aggregator.add_batch(labels, locations)
patchwise_output_tensor = aggregator.get_output_tensor()
plot_volume(
    patchwise_output_tensor.numpy().squeeze(),
    enhance=False,
    colors_path='GIFNiftyNet.ctbl',
)

  0%|          | 0/8 [00:00<?, ?batch/s]

/usr/local/lib/python3.7/dist-packages/torchio/data/inference/aggregator.py:229: RuntimeWarning: Medical image frameworks such as ITK do not support int64. Casting to int32...
  warnings.warn(message, RuntimeWarning)


Output()

The result is not as good as a full [GIF](https://spiral.imperial.ac.uk/bitstream/10044/1/30755/4/07086081.pdf) or [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/recon-all) parcellation but hey, it's 600 times faster!

## Test-time augmentation and uncertainty estimation

[Test-time augmentation (TTA)](https://www.nature.com/articles/s41598-020-61808-3) can be used to improve the results. We will apply some transform to the image, infer the segmentation on the transformed space and apply the inverse transform to bring the image to the result to the original space. Then, we can use majority voting to obtain a more robust segmentation result.

In [ ]:
def get_lr_remapping(table_path):
    import pandas as pd
    df = pd.read_csv(table_path, sep=' ', names=['Label', 'Name', *'RGBA'])
    mapping = {}
    for row in df.itertuples():
        if 'Left' in row.Name:
            mapping[row.Label] = df[df.Name == f'Right-{row.Name[5:]}'].Label.values[0]
        elif 'Right' in row.Name:
            mapping[row.Label] = df[df.Name == f'Left-{row.Name[6:]}'].Label.values[0]
    return mapping

In [ ]:
num_augmentations = 20 if device.type == 'cuda' else 3
results = []
remapping = get_lr_remapping('GIFNiftyNet.ctbl')

# If the image is flipped, labels should be changed accordingly
flip = tio.Compose((
        tio.RandomFlip(axes=['LR'], flip_probability=1),
        tio.RemapLabels(remapping),
    ),
    p=0.5,
)
resample = tio.OneOf({
    tio.RandomAffine(image_interpolation='nearest'): 0.75,
    tio.RandomElasticDeformation(image_interpolation='nearest'): 0.25,
})
augment = tio.Compose((flip, resample))
for _ in trange(num_augmentations):
    augmented = augment(preprocessed)
    input_tensor = augmented.t1.data[None].to(device)
    with torch.autocast(device.type):
        logits = model(input_tensor)
    output_tensor = logits.argmax(dim=1, keepdim=True).cpu()
    lm_temp = tio.LabelMap(tensor=torch.rand(1,1,1,1), affine=augmented.t1.affine)
    augmented.add_image(lm_temp, 'label')
    augmented.label.set_data(output_tensor[0])
    back = augmented.apply_inverse_transform(warn=True)
    results.append(back.label.data)
result = torch.stack(results).long()

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
tta_result_tensor = result.mode(dim=0).values  # majority voting
print(f'Using {num_augmentations} iterations:')
plot_volume(
    tta_result_tensor.numpy().squeeze(),
    enhance=False,
    colors_path='GIFNiftyNet.ctbl',
)
print('Using original image only:')
plot_volume(
    full_volume_output_tensor.numpy().squeeze(),
    enhance=False,
    colors_path='GIFNiftyNet.ctbl',
)

Using 20 iterations:


Output()

Using original image only:


Output()

The result using TTA should be more robust.

### Voxel-wise aleatoric uncertainty estimation

We can [estimate the aleatoric uncertainty](https://www.sciencedirect.com/science/article/pii/S0925231219301961) using our multiple results. We will use the technique from [Li et al.](https://link.springer.com/chapter/10.1007/978-3-319-59050-9_28): the uncertainty at each voxel is the percentage of predictions different to the most frequent prediction during our TTA.

In [ ]:
different = torch.stack([
    tensor != tta_result_tensor
    for tensor in results
])
uncertainty = different.float().mean(dim=0)
plot_volume(
    uncertainty.numpy().squeeze(),
    enhance=False,
)

Output()

As expected, the highest uncertainty values are in voxels on the structures boundaries.

## Conclusion

We have seen how to combine TorchIO and PyTorch hub to infer a full brain parcellation using patches of a 3D T1-weighted MRI and a pre-trained convolutional neural network, or the full volume.

We have also used TorchIO to perform test-time augmentation and estimate a voxel-wise aleatoric uncertainty of our prediction.

TorchIO is looking for feedback and contributors. Don't hesitate to [open an issue](https://github.com/fepegar/torchio/issues/new/choose) in the repository with questions or feature requests!